In [ ]:
1+1

In [ ]:
%load_ext autoreload
%autoreload 2

import os
import uproot
import matplotlib.pyplot as plt
import matplotlib.colors as colors
import matplotlib.cbook as cbook
from matplotlib.legend_handler import HandlerLine2D, HandlerTuple
import numpy as np
import pandas as pd
from decimal import Decimal
from scipy.stats import norm
from scipy.optimize import curve_fit
from scipy import stats
import datetime as dt
import scipy.optimize
import landau
from scipy.interpolate import CubicSpline, interp2d
from scipy.integrate import quad
from scipy.special import erf
from numpy import linalg

import importlib
from multiprocessing import Pool

from tqdm.auto import tqdm

# local imports
from lib.constants import *

In [ ]:
dosave = True
plt.rcParams.update({'font.size': 14})
plotqual = "data_"
savedir = "plots_11_14_23/%s" % plotqual

plottitle = "Run %i"
tpcnames = ["EE", "EW", "WE", "WW"]

datadir = "/pnfs/sbn/persistent/users/gputnam/calib-data/outputs/"
filedir = "/pnfs/sbn/persistent/users/gputnam/calib-data/Run1/"

# datadir = "/icarus/data/users/gputnam/DMCP2023G/calib-mc/"
# filedir = "/icarus/data/users/gputnam/DMCP2023G/calib-mc/"

isMC = False

savedata = False

In [ ]:
files = [f for f in os.listdir(filedir) if f.endswith(".df") and f.startswith("calib") and "anode" not in f
        and "sce" not in f and "old" not in f and "numi8" not in f and "bnb8" not in f]

if isMC:
    files = ["calib_mcnuphase2.df"]

files

In [ ]:
dfs = [pd.read_hdf(filedir + f) for f in files]

In [ ]:
offset = 0
for i,(fname, df) in enumerate(zip(files, dfs)):
    if len(df) == 0: continue
        
    df["itrk"] = df.index.get_level_values(0) + offset
    offset += df.index.get_level_values(0).max() + 1

In [ ]:
if not isMC:
    todelete = [c for c in dfs[0].columns if "true_" in c or "trueh_" in c]
    todelete += ["michelE", "closest_tdaughter"]
    for df in dfs:
        for c in todelete:
            del df[c]

In [ ]:
data = pd.concat(dfs, ignore_index=True)

In [ ]:
del dfs

In [ ]:
data.wire = data.wire.astype("int16")

In [ ]:
data["tpcEE"] = data.tpcE & (data.cryostat == 0)
data["tpcEW"] = ~data.tpcE & (data.cryostat == 0)
data["tpcWE"] = data.tpcE & (data.cryostat == 1)
data["tpcWW"] = ~data.tpcE & (data.cryostat == 1)

In [ ]:
data["itpc"] = -1
data.loc[data.tpcEE, "itpc"] = 0
data.loc[data.tpcEW, "itpc"] = 1
data.loc[data.tpcWE, "itpc"] = 2
data.loc[data.tpcWW, "itpc"] = 3

In [ ]:
data["thit"] = (data.time * tick_period - data.pandora_t0 - tanode*tick_period) / 1000.

In [ ]:
lifetime_file = datadir + "lifetimes_runArun1run2.txt"
data["lifetime"] = np.nan


if not isMC:
    with open(lifetime_file) as f:
        next(f)
        for line in f:
            dat = list(map(float, line.rstrip("\n").split(" ")))
            run = int(dat[0])
            data.loc[(data.run == run) & data.tpcEE, "lifetime"] = dat[1]
            data.loc[(data.run == run) & data.tpcEW, "lifetime"] = dat[2]
            data.loc[(data.run == run) & data.tpcWE, "lifetime"] = dat[3]
            data.loc[(data.run == run) & data.tpcWW, "lifetime"] = dat[4]
else:
    data.lifetime = 3e3

In [ ]:
data.columns

In [ ]:
del data["integral"]

In [ ]:
del data["width"]
del data["sumadc"]
# del data["dqdx_nocorr"]
del data["mint_tpcE"]
del data["maxt_tpcE"]
del data["mint_tpcW"]
del data["maxt_tpcW"]

In [ ]:
yz_ybin = np.linspace(-180, 130, 32)
yz_ylos = yz_ybin[:-1]
yz_yhis = yz_ybin[1:]
yz_ys = (yz_ylos + yz_yhis) / 2.

yz_zbin = np.linspace(-900, 900, 181)
yz_zlos = yz_zbin[:-1]
yz_zhis = yz_zbin[1:]
yz_zs = (yz_zlos + yz_zhis) / 2.

In [ ]:
data["ybin"] = np.searchsorted(yz_ybin, data.p_y.values) - 1

In [ ]:
data["zbin"] = np.searchsorted(yz_zbin, data.p_z.values) - 1

In [ ]:
data["scale_yz"] = np.nan

if not isMC:
#     scaleyz_file = datadir + "scaleYZ.txt"
    scaleyz_file = datadir + "P2_scaleYZ_RunA.txt"

    idx = []
    scales = []

    with open(scaleyz_file) as f:
        next(f)
        for line in f:
            dat = line.rstrip("\n").split("\t")
            tpc = dat[0]
            iy = int(dat[1])
            iz = int(dat[2])
            scale = float(dat[3])

            idx.append((tpcnames.index(tpc), iy, iz))
            scales.append(scale)

    scaledf = pd.DataFrame(scales, index=pd.MultiIndex.from_tuples(idx, names=["itpc", "ybin", "zbin"]), 
                           columns=["scale_yz"])
    dtmp = pd.merge(data[["itpc", "ybin", "zbin"]], scaledf, on=["itpc", "ybin", "zbin"], how="left")
    
    data["scale_yz"] = dtmp.scale_yz
    
    del dtmp

else:
    data.scale_yz = 1

In [ ]:
# Normalize by drift time
data["dqdx_normt"] = data.dqdx_nocorr * np.exp(data.thit / data.lifetime)
data["dqdx_normyz"] = data.dqdx_nocorr / data.scale_yz
data["dqdx_normed"] = data.dqdx_normt / data.scale_yz

In [ ]:
def fidYZ(data, iny=20, inz=100):
    ymax = 134
    ymin = -180
    
    zmin = -900
    zmax = 900
    
    fid = (data.p_y > ymin + iny) & (data.p_y < ymax - iny)\
        & (data.p_z < zmax - inz) & (data.p_z > zmin + inz)
    
    if not isMC:
        # Cut out some problem regions in the detector
        fid = fid & (np.abs(data.p_z) > 10)

        # TPC EW
        bad_tpcEW = data.tpcEW & (data.p_z > 700) & (data.p_y < 0)

        # TPC WW
        bad_tpcWW = data.tpcWW & (data.p_y > 80) & (data.p_z < 0)

        fid = fid & ~bad_tpcEW & ~bad_tpcWW
    
    return fid

In [ ]:
data["fid"] = fidYZ(data) & (data.thit > 100) & (data.thit < 900)

In [ ]:
isMC

In [ ]:
# Constants
if isMC:
    LAr_density_gmL = 1.389875
else:
    LAr_density_gmL = 1.3926
    
mass_electron = 0.5109989461 # MeV https://pdg.lbl.gov/2020/listings/rpp2020-list-K-plus-minus.pdf
mass = 105.6583745 # MeV https://pdg.lbl.gov/2020/listings/rpp2020-list-muon.pdf
Ival = 188.0e-6 if isMC else 197.0e-6
Zval = 18.0
Aval = 39.948
# Kfactor = 0.307075

Ar_molar_mass = 39.9623
Ar_ZA = 18. / Ar_molar_mass
Relec = 2.817940 * 1e-13
mole = 6.0221409*1e23
Kfactor = 4*np.pi*mole*Relec**2*mass_electron # 0.307075


In [ ]:
def Calc_MEAN_DEDX(T):
    gamma = (mass+T)/mass
    beta = np.power(1.0-np.power(gamma,-2.0),0.5)
    Wmax = (2.0*mass_electron*np.power(beta,2.0)*np.power(gamma,2.0))/(1.0+2.0*gamma*(mass_electron/mass)+np.power(mass_electron/mass,2.0))

    # Medium energy 
    dens_factor = 2.0*np.log(10)*np.log10(beta*gamma)-5.2146+0.19559*np.power(3.0-np.log10(beta*gamma),3.0)
    # low energy
    dens_factor[np.log10(beta*gamma) < 0.2] = 0.
    dens_factor[beta < 1e-6] = 0.
    # high energy
    dens_factor[np.log10(beta*gamma) > 3.0] = (2.0*np.log(10)*np.log10(beta*gamma)-5.2146)[np.log10(beta*gamma) > 3.0]
    dEdx_mean = LAr_density_gmL*Kfactor*(Zval/Aval)*np.power(beta,-2.0)*(0.5*np.log(2.0*mass_electron*np.power(beta,2.0)*np.power(gamma,2.0)*Wmax*np.power(Ival,-2.0))-np.power(beta,2.0)-dens_factor/2.0)

    return dEdx_mean

def Calc_MPV_DEDX(thick, T):
    gamma = (mass+T)/mass
    beta = np.power(1.0-np.power(gamma,-2.0),0.5)
    Wmax = (2.0*mass_electron*np.power(beta,2.0)*np.power(gamma,2.0))/(1.0+2.0*gamma*(mass_electron/mass)+np.power(mass_electron/mass,2.0))

    # Medium energy 
    dens_factor = 2.0*np.log(10)*np.log10(beta*gamma)-5.2146+0.19559*np.power(3.0-np.log10(beta*gamma),3.0)
    # low energy
    dens_factor[np.log10(beta*gamma) < 0.2] = 0.
    # high energy
    dens_factor[np.log10(beta*gamma) > 3.0] = (2.0*np.log(10)*np.log10(beta*gamma)-5.2146)[np.log10(beta*gamma) > 3.0]
    xi = (Kfactor/2.0)*(Zval/Aval)*np.power(beta,-2.0)*LAr_density_gmL*thick
    dEdx_MPV = xi*(np.log((2.0*mass_electron*np.power(beta*gamma,2.0))/Ival)+np.log(xi/Ival)+0.200-np.power(beta,2.0)-dens_factor)/thick
    return dEdx_MPV


def Calc_RR_points(KE, dRR=0.01):
    thisKE = KE
    KE_points = [thisKE]
    RR_points = [0.]

    while thisKE > 0.0:
        deltaKE = Calc_MEAN_DEDX(np.array([thisKE])) * dRR
        RR_points.append(RR_points[-1] + dRR)
        thisKE -= deltaKE[0]
        KE_points.append(thisKE)

    KE_points = np.array(list(reversed(KE_points[:-1])))
    RR_points = np.array(RR_points[:-1])

    return KE_points, RR_points

KE_points_max = 1000.
KE_points, RR_points = Calc_RR_points(KE_points_max)

RR2KE = CubicSpline(RR_points, KE_points)


In [ ]:
def Calc_Q2KE_points(KE, recomb, dQ0=500, mass=mass, z=1):
    thisKE = KE
    KE_points = []
    Q_points = []
    while thisKE > 0.0:
        dEdx = Calc_MEAN_DEDX(np.array([thisKE]))
        dQdx = recomb(dEdx)
        dx = dQ0/dQdx[0]
        deltaKE = dEdx * dx
        dQ = dQdx*dx
        Q_points.append(dQ)
        thisKE -= deltaKE[0]
        KE_points.append(thisKE)

    KE_points = np.flip(np.array(KE_points[:-1]), axis=0)
    Q_points = np.cumsum(np.flip(np.array(Q_points[:-1]), axis=0), axis=0)

    return Q_points.squeeze().squeeze(), KE_points


In [ ]:
def RR2MeandEdx(RR):
    return Calc_MEAN_DEDX(RR2KE(RR))

In [ ]:
# ArgoNeuT params
MODA = 0.930
MODB = 0.212
Wion = 1e3 / 4.237e7
if isMC: # MC Efield
    Efield = 0.494
else: # data efield
    Efield = 0.4926
print(Efield)
    
def recombination(dEdx, A=MODA, B=MODB, E=Efield):
    alpha = A
    beta = B / (LAr_density_gmL * E)

    dQdx = np.log(alpha + dEdx*beta) / (Wion * beta)
    return dQdx

def recombination_cor(dQdx, A=MODA, B=MODB, E=Efield):
    alpha = A
    beta = B / (LAr_density_gmL * E)
        
    dEdx = (np.exp(dQdx*Wion*beta)- alpha) / beta
        
    return dEdx
    

In [ ]:
rpt = data.groupby("itrk").itrk.count()

In [ ]:
endp_ind = data.groupby("itrk").rr.idxmin()

In [ ]:
data["is_stopping"] = True

In [ ]:
wsorted_df = data[["itrk", "wire", "itpc"]].sort_values(["itrk", "itpc", "wire"])

In [ ]:
wdiff = wsorted_df.groupby(["itrk", "itpc"]).wire.diff()

In [ ]:
data["wdiff"] = wdiff

In [ ]:
missingE = np.maximum((np.abs(data.wdiff) - 1).fillna(0), 0)*data.pitch*RR2MeandEdx(data.rr)

In [ ]:
data["fid_missE"] = missingE
data.loc[~data.fid, "fid_missE"] = np.nan

In [ ]:
def emb_beta(phi, beta90=0.203, R=1.25):
    phirad = phi*np.pi/180
    return beta90 / np.sqrt(np.sin(phirad)**2 + np.cos(phirad)**2/R**2)

In [ ]:
if isMC:
    beta = 0.212
    alpha = 0.93
    gain = 1/0.01265 # MCNuPhase2
    gain = 78.1 # MCNuPhase2
else:
    # Nominal Fit
    betaf = emb_beta
    beta = betaf(82.5)
    alpha = 0.906
    gain = 75.1
    
    # Nominal, updated density
    betaf = emb_beta
    beta = betaf(82.5, 0.205)
    alpha = 0.903
    gain = 74.9
    
    # No diffuson fit
#     betaf = lambda phi: emb_beta(phi, 0.204, 1.25)
#     beta = betaf(82.5)
#     alpha = 0.924
#     gain = 75.4

In [ ]:
CALS = {}

# Divide out the gain in the input files
if not isMC:
    with open(datadir + "tpc_ratio_Run1.txt") as f:
        for line in f:
            TPC, CAL = line.rstrip("\n").split(" ")
            CALS[TPC] = float(CAL)
else:
    CALS["MC"] = 1.

CALS

In [ ]:
data["phi"] = np.arccos(np.abs(data.dir_x))*180/np.pi

In [ ]:
data["thxw"] = np.abs(np.arctan(data.dir_x*data.pitch/0.3)*180/np.pi)

In [ ]:
if not isMC:
    data["dedx"] = recombination_cor(data.dqdx_normed*CALS["EE"]*gain, 
                                                     A=alpha, B=betaf(data.phi))
    data.loc[data.tpcEW, "dedx"] = recombination_cor(data.dqdx_normed[data.tpcEW]*CALS["EW"]*gain, 
                                                     A=alpha, B=betaf(data.phi))
    data.loc[data.tpcWE, "dedx"] = recombination_cor(data.dqdx_normed[data.tpcWE]*CALS["WE"]*gain, 
                                                     A=alpha, B=betaf(data.phi))
    data.loc[data.tpcWW, "dedx"] = recombination_cor(data.dqdx_normed[data.tpcWW]*CALS["WW"]*gain, 
                                                     A=alpha, B=betaf(data.phi))
else:
    data["dedx"] = recombination_cor(data.dqdx_normed*CALS["MC"]*gain, A=alpha, B=beta)

In [ ]:
rrbins = np.linspace(0, 100, 101)
rrs = (rrbins[1:] + rrbins[:-1]) / 2.

In [ ]:
whenplt = data.is_stopping & data.fid & (data.pitch < 1)

In [ ]:
data.thit[whenplt].mean(), data.pitch[whenplt].mean()

In [ ]:
_ = plt.hist2d(data.rr[whenplt], data.dedx[whenplt], bins=[rrbins, np.linspace(1, 6, 41)], rasterized=True)
plt.xlabel("Residual Range [cm]")
plt.ylabel("Calibrated dE/dx [MeV/cm]")

plt.plot(rrs, RR2MeandEdx(rrs), color="red", label="Predicted MPV dE/dx")
plt.legend(frameon=False, labelcolor="white")
plt.tight_layout()

if dosave: 
    plt.savefig(savedir + "dedx_rr_vpredicted.pdf")

In [ ]:
dedxdf_tosave = data.loc[data.fid, ["dedx", "rr"]]

In [ ]:
dedxdf_tosave

In [ ]:
if dosave:
    dedxdf_tosave.to_hdf(savedir + "muon_dedx.df", key="dedx")

In [ ]:
# data

In [ ]:
data["ke"] = data.dedx * data.pitch
data.loc[~data.fid, "ke"] = np.nan

In [ ]:
data["fid_rr"] = data.rr
data.loc[~data.fid, "fid_rr"] = np.nan

In [ ]:
data["fid_pitch"] = data.pitch
data.loc[~data.fid, "fid_pitch"] = np.nan

In [ ]:
data["fid_phi"] = data.phi
data.loc[~data.fid, "fid_phi"] = np.nan

In [ ]:
data["fid_thxw"] = data.thxw
data.loc[~data.fid, "fid_thxw"] = np.nan

In [ ]:
data["fid_true_e"] = data.trueh_e if isMC else np.nan
data.loc[~data.fid, "fid_true_e"] = np.nan

In [ ]:
mugroup = data.groupby("itrk")

In [ ]:
muonCalE = mugroup.ke.sum()

In [ ]:
muonTrueE = mugroup.fid_true_e.sum()

In [ ]:
muonMissE = mugroup.fid_missE.sum()

In [ ]:
mupitch = mugroup.fid_pitch.mean()

In [ ]:
muphi = mugroup.fid_phi.mean()

In [ ]:
muthxw = mugroup.fid_thxw.mean()

In [ ]:
# Q tip KE

In [ ]:
data["qtip"] = data.dqdx_normed*data.pitch*(data.rr < 3)*gain
data.loc[~data.fid, "qtip"] = np.nan

In [ ]:
data["elen"] = data.ke*(data.rr >= 3)

In [ ]:
muontipQ = mugroup.qtip.sum()

In [ ]:
Q2KE = CubicSpline(*Calc_Q2KE_points(200, lambda dEdx: recombination(dEdx, alpha, beta)), 
                   extrapolate=False)

In [ ]:
muontipE = Q2KE(muontipQ.values)

In [ ]:
muonqtipE = muontipE + mugroup.elen.sum()

In [ ]:
mu_max_rr = mugroup.fid_rr.max()

In [ ]:
mu_max_rr_data = np.repeat(mu_max_rr, rpt).reset_index()
data["max_rr"] = mu_max_rr_data.fid_rr.values

In [ ]:
data["badhit"] = (data.rr < data.max_rr) & ((~data.fid) | (data.ke > 100))

In [ ]:
whenmu = mugroup.is_stopping.all() & ~mugroup.badhit.any() & (mu_max_rr > 50) #& (muonMissE / muonCalE < 0.05)'calib_fitwvfks_fixallgain.df'

In [ ]:
whenmu.sum()

In [ ]:
muonRangeE = RR2KE(mu_max_rr)

In [ ]:
_ = plt.hist((muonMissE / muonCalE)[~mugroup.badhit.any() & (mu_max_rr > 100) ], bins=np.linspace(0, 1, 21))

In [ ]:
#_ = plt.hist(data.ke, bins=np.linspace(0, 100, 21))
#plt.yscale("log")

In [ ]:
def gauss(X, A, mu, sigma):
    return A * np.exp(-(X-mu)**2 / (2*sigma**2))

def double_gauss(X, A1, mu1, sigma1, A2, mu2, sigma2):
    return A1 * np.exp(-(X-mu1)**2 / (2*sigma1**2)) + A2 * np.exp(-(X-mu2)**2 / (2*sigma2**2)) 

def lorentzian(X, A, mu, sigma):
    return (A/np.pi)*((sigma/2) / ((X - mu)**2 + (sigma/2)**2))

In [ ]:
def energy_comparison_plot(plt, v, bins, text="left", fit_double=True, energy1="calo", energy2="range"):
    N,bins = np.histogram(v, bins=bins)
    centers = (bins[1:] + bins[:-1]) / 2
    XS = np.linspace(bins[0], bins[-1], 1000)
    
    plt.errorbar(centers[N>0], N[N>0], yerr=np.sqrt(N)[N>0], 
                 linestyle="none", marker=".", color="black")
    
    ymax = N.max()

    if fit_double:
        p0 = [
            ymax/2, 0.0, 0.05,
            ymax/10, 0.15, 0.1
        ]

        popt, perr = curve_fit(double_gauss, centers, N, sigma=np.maximum(1, np.sqrt(N)), p0=p0, maxfev=10_000)

        p1 = popt[:3]
        p2 = popt[-3:]
        if p1[0] < p2[0]:
            p1 = popt[-3:]
            p2 = popt[:3]

        peak_val = np.max(double_gauss(XS, *popt))
        peak_ind = np.argmax(double_gauss(XS, *popt))
        half_lo = XS[np.argmin(np.abs(peak_val/2 - double_gauss(XS[:peak_ind], *popt)))]
        half_hi = XS[peak_ind + np.argmin(np.abs(peak_val/2 - double_gauss(XS[peak_ind:], *popt)))]    
        FWHM = half_hi - half_lo

        plt.plot(XS, double_gauss(XS, *popt))
        plt.plot(XS, gauss(XS, *p1), color="gray", linestyle=":")
        plt.plot(XS, gauss(XS, *p2), color="gray", linestyle="--")        
    else:
        p0 = [
            ymax, 0.0, 0.05,
        ]

        popt, perr = curve_fit(gauss, centers, N, p0=p0, maxfev=10_000)

        p1 = popt[:3]

        peak_val = np.max(gauss(XS, *popt))
        peak_ind = np.argmax(gauss(XS, *popt))
        half_lo = XS[np.argmin(np.abs(peak_val/2 - gauss(XS[:peak_ind], *popt)))]
        half_hi = XS[peak_ind + np.argmin(np.abs(peak_val/2 - gauss(XS[peak_ind:], *popt)))]    
        FWHM = half_hi - half_lo

        plt.plot(XS, gauss(XS, *popt))
    
    trans = plt.gca().get_yaxis_transform()
    
    if text == "left":
        x_lo = 0.025
        x_hi = 0.35
        x_center = 0.1
        x_fwhm = 0.7
    else:
        x_lo = 0.6
        x_hi = 0.975
        x_center = 0.7
        x_fwhm = 0.05
    

    
    if fit_double:
        plt.axhline(ymax*0.8, x_lo, x_hi, color="gray", linestyle=":")    
        plt.text(x_lo, ymax*0.85, "$\\mu_1: %.1f$%%, $\\sigma_1: %.1f$%%" % (p1[1]*100, np.abs(p1[2])*100),
            transform=trans, fontsize=12)
        
        plt.axhline(ymax*0.6, x_lo, x_hi, color="gray", linestyle="--")    
        
        plt.text(x_lo, ymax*0.65, "$\\mu_2: %.1f$%%, $\\sigma_2: %.1f$%%" % (p2[1]*100, np.abs(p2[2])*100),
                transform=trans, fontsize=12)
    
        plt.text(x_center, ymax*0.5, "$N_1/N_2: %.2f$" % (p1[0] / p2[0]),
                transform=trans, fontsize=12)
    else:
        plt.text(x_lo, ymax*0.85, "$\\mu: %.1f$%%, $\\sigma: %.1f$%%" % (p1[1]*100, np.abs(p1[2])*100),
            transform=trans, fontsize=12)
        
    plt.text(x_fwhm, 0.9, "FWHM: %.0f%%" % (FWHM*100), fontsize=12, transform=plt.gca().transAxes)
    
    if fit_double:
        plt.text(0.03, 0.875, "Double Gaussian Fit", fontsize=12, transform=plt.gca().transAxes)
    else:
        plt.text(0.03, 0.875, "Single Gaussian Fit", fontsize=12, transform=plt.gca().transAxes)
        
    if isMC:
        plt.text(x_fwhm, 0.6, "ICARUS\nMC", fontsize=20, transform=plt.gca().transAxes)
    else:
        plt.text(x_fwhm, 0.6, "ICARUS\nData", fontsize=20, transform=plt.gca().transAxes)
    
    plt.xlabel("$(E_\\mathrm{%s} - E_\\mathrm{%s}) / E_\\mathrm{%s}$" % (energy1, energy2, energy2))

    plt.ylabel("Muon Like Tracks")

In [ ]:
var = (muonCalE - muonRangeE) / muonRangeE
bins = (np.linspace(-1, 1, 41) + 0.025) / 2
centers = (bins[:-1] + bins[1:]) / 2.

In [ ]:
PITCHS = [0.3, 0.4]
PLOS = PITCHS[:-1]
PHIS = PITCHS[1:]

In [ ]:
for ifig, (plo, phi) in enumerate(zip(PLOS, PHIS)):
    plt.figure(ifig)
    thiswhen = whenmu &\
        (mupitch < phi) & (mupitch > plo) &\
        (muphi > 70) & (muphi < 85) &\
        (muthxw > 5) & (muthxw < 20)

    energy_comparison_plot(plt, var[thiswhen], bins)
    
    if dosave: plt.savefig(savedir + "calE_v_rangeE.pdf")

In [ ]:
var = (muonqtipE - muonRangeE) / muonRangeE
bins = (np.linspace(-1, 1, 41) + 0.025) / 2
centers = (bins[:-1] + bins[1:]) / 2.

In [ ]:
for ifig, (plo, phi) in enumerate(zip(PLOS, PHIS)):
    plt.figure(ifig)
    thiswhen = whenmu &\
        (mupitch < phi) & (mupitch > plo) &\
        (muphi > 70) & (muphi < 85) &\
        (muthxw > 5) & (muthxw < 20)

    energy_comparison_plot(plt, var[thiswhen], bins)

    if ifig == 0 and dosave: plt.savefig(savedir + "calE_tip_v_rangeE.pdf")

In [ ]:
data["p_z_abs"] = np.abs(data.p_z)

In [ ]:
var = (muonqtipE - muonRangeE + muonMissE) / muonRangeE
bins = ((np.linspace(-1, 1, 41) + 0.025) / 4)[:-1]
centers = (bins[:-1] + bins[1:]) / 2

In [ ]:
for ifig, (plo, phi) in enumerate(zip(PLOS, PHIS)):
    plt.figure(ifig)
    thiswhen = whenmu &\
        (mupitch < phi) & (mupitch > plo) &\
        (muphi > 70) & (muphi < 85) &\
        (muthxw > 5) & (muthxw < 20) &\
        (mugroup.p_z_abs.min() > 150)

    energy_comparison_plot(plt, var[thiswhen], bins)

    if ifig == 0 and dosave: plt.savefig(savedir + "calE_tip_miss_v_rangeE.pdf")

In [ ]:
for ifig, (plo, phi) in enumerate(zip(PLOS, PHIS)):
    plt.figure(ifig)
    thiswhen = whenmu &\
        (mupitch < phi) & (mupitch > plo) &\
        (muphi > 70) & (muphi < 85) &\
        (muthxw > 5) & (muthxw < 20) &\
        (mugroup.p_z_abs.min() > 150)

    energy_comparison_plot(plt, var[thiswhen], bins, fit_double=False)

    if ifig == 0 and dosave: plt.savefig(savedir + "calE_tip_miss_v_rangeE_singlegauss.pdf")

In [ ]:
var = (muonRangeE - muonTrueE - muonMissE) / (muonTrueE + muonMissE)
bins = ((np.linspace(-1, 1, 41) + 0.025) / 4)[:-1]
centers = (bins[:-1] + bins[1:]) / 2

In [ ]:
for ifig, (plo, phi) in enumerate(zip(PLOS, PHIS)):
    plt.figure(ifig)
    thiswhen = whenmu &\
        (mupitch < phi) & (mupitch > plo) &\
        (muphi > 70) & (muphi < 85) &\
        (muthxw > 5) & (muthxw < 20) &\
        (mugroup.p_z_abs.min() > 150)

    energy_comparison_plot(plt, var[thiswhen], bins, fit_double=True, energy1="range", energy2="true")

    if ifig == 0 and dosave: plt.savefig(savedir + "rangeE_v_trueE.pdf")

In [ ]:
# var = (muonTrueE - muonRangeE + muonMissE) / muonRangeE
var = (muonqtipE - muonTrueE) / muonTrueE
bins = ((np.linspace(-1, 1, 41) + 0.025) / 4)[:-1]
centers = (bins[:-1] + bins[1:]) / 2

In [ ]:
for ifig, (plo, phi) in enumerate(zip(PLOS, PHIS)):
    plt.figure(ifig)
    thiswhen = whenmu &\
        (mupitch < phi) & (mupitch > plo) &\
        (muphi > 70) & (muphi < 85) &\
        (muthxw > 5) & (muthxw < 20) &\
        (mugroup.p_z_abs.min() > 150)

    energy_comparison_plot(plt, var[thiswhen], bins, fit_double=True, energy1="calo", energy2="true")

    if ifig == 0 and dosave: plt.savefig(savedir + "caloE_v_trueE.pdf")